In [2]:
import spotipy
import billboard
import time

from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
import time
import pandas as pd
import numpy as np
from spotipy.oauth2 import SpotifyClientCredentials
cid = "b3c5b350c6cb43c2bdb403eb3b896eba"
secret = "fa1aa37ba9c943d3b4e2cadc8fd4818f"
client_credentials_manager = SpotifyClientCredentials(client_id=cid, client_secret=secret)
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)


In [3]:
def getTrackIDs(user, playlist_id):
    ids = []
    playlist = sp.user_playlist(user, playlist_id)
    for item in playlist['tracks']['items']:
        track = item['track']
        ids.append(track['id'])
    return ids

In [4]:
def getTrackIDsFromAlbum(album_id):
    ids2 = []
    #playlist = sp.user_playlist(user, playlist_id)
    album = sp.album(album_id)
    for item in album['tracks']['items']:
        ids2.append(item['id'])
    return ids2

In [5]:
def getTrackFeatures(id):
      meta = sp.track(id)
      features = sp.audio_features(id)

      # meta
      song_id = id
      name = meta['name']
      artist = meta['album']['artists'][0]['name']
      length = meta['duration_ms']
      popularity = meta['popularity']

      # features
      acousticness = features[0]['acousticness']
      danceability = features[0]['danceability']
      energy = features[0]['energy']
      instrumentalness = features[0]['instrumentalness']
      liveness = features[0]['liveness']
      loudness = features[0]['loudness']
      speechiness = features[0]['speechiness']
      tempo = features[0]['tempo']
      time_signature = features[0]['time_signature']
      valence = features[0]['valence']
      mode = features[0]['mode']
      weeks = 1
      date = 1
      target = 1

      track = {"name" : name, "artist" : artist, "song_id" : song_id, "length" : length, "popularity" : popularity, 
               "danceability" : danceability, "acousticness" : acousticness, "energy" : energy, "instrumentalness" : instrumentalness, 
               "liveness" : liveness, "loudness" : loudness, "speechiness" : speechiness, "tempo" : tempo, "time_signature" : time_signature, 
               "valence" : valence, "mode" : mode, "weeks" : weeks, "date" : date, "target" : target}
      return track

In [6]:
def getTrackIdFromBillboard(name, artist):
    results = sp.search(q="track:" + name + " artist:" + artist)
    items = results["tracks"]["items"]
    id = "NONE"
    if (len(items) > 0):
        id = items[0]["id"]
    return id

In [ ]:
def occurencesInNewList(list, artist):
    occurences = 1
    for dict in list:
        if dict["artist"] == artist:
            occurences += 1
    return occurences

In [7]:
def getChartDataFrame(chart, fullDataFrame):
    tracks = []
    for entry in chart.entries:
        id = getTrackIdFromBillboard(entry.title, entry.artist)
        if (id != "NONE"):
            if (entry.artist in fullDataFrame.values) and (not any(dic['artist'] == entry.artist for dic in tracks)):
                fullDataFrame.loc[fullDataFrame['artist'] == entry.artist, 'weeks'] += 1
            if (id not in fullDataFrame.values) and (entry.isNew):
                track = getTrackFeatures(id)
                track["date"] = chart.date
                tracks.append(track)
    df = pd.DataFrame(tracks)
    fullDataFrame = fullDataFrame.append(df)
    return fullDataFrame

In [13]:
def gatherBillboardData():
    chart = billboard.ChartData('hot-100')
    i = 0
    fullDataFrame = pd.DataFrame(columns = ['name', 'artist', 'song_id', 'length', 'popularity', 'danceability', 'acousticness', 'energy', 'instrumentalness', 'liveness', 'loudness', 'speechiness', 'tempo', 'time_signature', 'valence', 'mode', 'weeks', 'date'])
    start = time.clock()
    while (chart.previousDate):
        start = time.clock()
        chart = billboard.ChartData('hot-100', chart.previousDate)
        fullDataFrame = getChartDataFrame(chart, fullDataFrame)
        end = time.clock()
        print("total time: " + str(end-start))
        i += 1
        if (i == 4):
            break
    end = time.clock()
    print("total time: " + str(end-start))
    return fullDataFrame

In [14]:
dataFrame = gatherBillboardData()
#dataFrame.to_csv('billboard-data.csv')

C:\Users\eskil\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: DeprecationWarning: time.clock has been deprecated in Python 3.3 and will be removed from Python 3.8: use time.perf_counter or time.process_time instead
  """
C:\Users\eskil\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: DeprecationWarning: time.clock has been deprecated in Python 3.3 and will be removed from Python 3.8: use time.perf_counter or time.process_time instead
  import sys
C:\Users\eskil\Anaconda3\lib\site-packages\ipykernel_launcher.py:10: DeprecationWarning: time.clock has been deprecated in Python 3.3 and will be removed from Python 3.8: use time.perf_counter or time.process_time instead
  # Remove the CWD from sys.path while we load stuff.


total time: 11.7853597000003
total time: 11.263204700000642
total time: 12.922648499999923
total time: 11.179649900000186
total time: 11.180882000000565


C:\Users\eskil\Anaconda3\lib\site-packages\ipykernel_launcher.py:15: DeprecationWarning: time.clock has been deprecated in Python 3.3 and will be removed from Python 3.8: use time.perf_counter or time.process_time instead
  from ipykernel import kernelapp as app


In [16]:
dataFrame.head(40)
#dataFrame.loc[(dataFrame['artist'] == "Polo G")]


,acousticness,artist,danceability,date,energy,instrumentalness,length,liveness,loudness,mode,name,popularity,song_id,speechiness,target,tempo,time_signature,valence,weeks
0,0.50400,Kelsea Ballerini,0.672,2020-10-24,0.552,0.000000,152440,0.0636,-6.375,1,hole in the bottle - ballerini album version,70,1f6BZccfonUYohfGJIeBR1,0.0316,1.0,105.020,4,0.5940,1
1,0.59800,Clairo,0.744,2020-10-24,0.619,0.003720,188386,0.2310,-9.805,1,Sofia,88,7B3z0ySL9Rr0XvZEAjWZzM,0.0390,1.0,112.997,4,0.6410,1
0,0.13100,Shawn Mendes,0.333,2020-10-17,0.637,0.000018,172692,0.1490,-4.904,0,Wonder,92,5KCbr5ndeby4y4ggthdiAb,0.0581,1.0,139.898,4,0.1320,1
1,0.01010,Why Don't We,0.606,2020-10-17,0.900,0.000000,216504,0.0750,-4.504,1,Fallin’ (Adrenaline),80,5GhJIMWAw0uMLgkdbt6uMz,0.1190,1.0,133.963,4,0.2330,1
2,0.00668,BLACKPINK,0.658,2020-10-17,0.733,0.000000,192760,0.1180,-4.756,1,Lovesick Girls,86,4Ws314Ylb27BVsvlZOy30C,0.0586,1.0,127.977,4,0.4640,1
3,0.16800,Bryson Tiller,0.605,2020-10-17,0.489,0.000002,170053,0.1870,-7.234,0,Always Forever,72,1LV5cAo02H8h5YlZNcjULM,0.1740,1.0,147.341,4,0.3260,1
4,0.02790,Bryson Tiller,0.374,2020-10-17,0.577,0.000006,179976,0.5620,-7.085,1,Years Go By,72,4SVTus5gJc5cfkFZ8ELK1p,0.3020,1.0,184.201,4,0.0658,1
0,0.12800,Polo G,0.616,2020-10-10,0.617,0.000000,177272,0.1770,-8.431,1,Epidemic,81,170DNUSnQRp1tGpppF26pA,0.5200,1.0,175.929,4,0.4760,2
1,0.00021,Machine Gun Kelly,0.533,2020-10-10,0.858,0.000000,205158,0.1480,-4.212,1,bloody valentine,80,6mADjHs6FXdroPzEGW6KVJ,0.0766,1.0,160.057,4,0.5060,1
2,0.08450,Ne-Yo,0.656,2020-10-10,0.647,0.000087,185934,0.3150,-7.584,1,"U 2 Luv (Remix feat. Jeremih, Queen Naija, & L...",54,4ngz3vlQcFj3aNnTNikoRv,0.1080,1.0,181.884,4,0.6860,1


In [ ]:
name = "fallin'"
artist = "Why Don't We"
results = sp.search(q="track:" + name + " artist:" + artist)
items = results["tracks"]["items"]
if (len(items) == 0):
    print("no items")
else:
    id = items[0]["id"]
    print(id)

In [ ]:
morn = np.array([{"name" : "Eskil", "age" : 23, "goals": 10},
                {"name" : "Arne", "age" : 15, "goals" : 30},
                {"name" : "Eskil", "age": 16, "goals" : 40}])
morn[1]["name"]

In [ ]:
hei = np.array([["sang1", "artist1", 10], 
               ["sang1", "artist2", 20]])
morn = np.array([{"name" : "Eskil", "age" : 23, "goals": 10},
                {"name" : "Arne", "age" : 15, "goals" : 30},
                {"name" : "Eskil", "age": 16, "goals" : 40}])
morn[1]["age"]



        
    